# A full business solution

## Now we will take our project from Day 1 to the next level

### BUSINESS CHALLENGE:

Create a product that builds a Brochure for a company to be used for prospective clients, investors and potential recruits.

We will be provided a company name and their primary website.

See the end of this notebook for examples of real-world business applications.

And remember: I'm always available if you have problems or ideas! Please do reach out.

In [1]:
# imports
# If these fail, please check you're running from an 'activated' environment with (llms) in the command prompt

import os
import json
from dotenv import load_dotenv
from IPython.display import Markdown, display, update_display
from scraper import fetch_website_links, fetch_website_contents
from openai import OpenAI

In [10]:
# Initialize and constants

import os
import httpx
from dotenv import load_dotenv

load_dotenv(override=True)
api_key = os.getenv('XAI_API_KEY')
reasoning_model = "grok-4-1-fast-reasoning"
non_reasoning_model = "grok-4-1-fast-non-reasoning"

if not api_key:
    print("No API key was found - please head over to the troubleshooting notebook in this folder to identify & fix!")
elif not api_key.startswith("xai-"):
    print("An API key was found, but it doesn't start xai-; please check you're using the right key - see troubleshooting notebook")
elif api_key.strip() != api_key:
    print("An API key was found, but it looks like it might have space or tab characters at the start or end - please remove them - see troubleshooting notebook")
else:
    print("API key found and looks good so far!")

openai = OpenAI(
    api_key=api_key,
    base_url="https://api.x.ai/v1",
    timeout=httpx.Timeout(3600.0), # Override default timeout with longer timeout for reasoning models
)

API key found and looks good so far!


In [3]:
links = fetch_website_links("https://www.dintaifung.com.tw/store.php")
links

['index.php',
 'about.php',
 'news.php',
 'store.php',
 'food.php',
 'care.php',
 'contact.php',
 'http://www.dintaifung.tw',
 'https://www.104.com.tw/jobbank/custjob/index.php?r=cust&j=4c70436d485c3e6848323c1d1d1d1d5f2443a363189j99',
 'http://www.dintaifung.tw/Queue/?type=3',
 'store.php',
 'store_world.php',
 'store.php?cid=1',
 'http://www.dintaifung.tw/Queue/?type=3',
 '#app',
 'https://itunes.apple.com/tw/app/%E9%BC%8E%E6%B3%B0%E8%B1%90/id1108359809?mt=8',
 'https://play.google.com/store/apps/details?id=com.dtf.orderapp&hl=zh_TW']

## First step: Have GPT-5-nano figure out which links are relevant

### Use a call to gpt-5-nano to read the links on a webpage, and respond in structured JSON.  
It should decide which links are relevant, and replace relative links such as "/about" with "https://company.com/about".  
We will use "one shot prompting" in which we provide an example of how it should respond in the prompt.

This is an excellent use case for an LLM, because it requires nuanced understanding. Imagine trying to code this without LLMs by parsing and analyzing the webpage - it would be very hard!

Sidenote: there is a more advanced technique called "Structured Outputs" in which we require the model to respond according to a spec. We cover this technique in Week 8 during our autonomous Agentic AI project.

In [4]:
link_system_prompt = """
You are provided with a list of links found on a webpage.
You are able to decide which of the links would be most relevant to include in a brochure about the company,
such as links to an About page, or a Company page, or Careers/Jobs pages.
You should respond in JSON as in this example:

{
    "links": [
        {"type": "about page", "url": "https://full.url/goes/here/about"},
        {"type": "careers page", "url": "https://another.full.url/careers"}
    ]
}
"""

In [5]:
def get_links_user_prompt(url):
    user_prompt = f"""
Here is the list of links on the website {url} -
Please decide which of these are relevant web links for a brochure about the company, 
respond with the full https URL in JSON format.
Do not include Terms of Service, Privacy, email links.

Links (some might be relative links):

"""
    links = fetch_website_links(url)
    user_prompt += "\n".join(links)
    return user_prompt

In [6]:
print(get_links_user_prompt("https://www.dintaifung.com.tw/store.php"))


Here is the list of links on the website https://www.dintaifung.com.tw/store.php -
Please decide which of these are relevant web links for a brochure about the company, 
respond with the full https URL in JSON format.
Do not include Terms of Service, Privacy, email links.

Links (some might be relative links):

index.php
about.php
news.php
store.php
food.php
care.php
contact.php
http://www.dintaifung.tw
https://www.104.com.tw/jobbank/custjob/index.php?r=cust&j=4c70436d485c3e6848323c1d1d1d1d5f2443a363189j99
http://www.dintaifung.tw/Queue/?type=3
store.php
store_world.php
store.php?cid=1
http://www.dintaifung.tw/Queue/?type=3
#app
https://itunes.apple.com/tw/app/%E9%BC%8E%E6%B3%B0%E8%B1%90/id1108359809?mt=8
https://play.google.com/store/apps/details?id=com.dtf.orderapp&hl=zh_TW


In [11]:
def select_relevant_links(url):
    response = openai.chat.completions.create(
        model=non_reasoning_model,
        messages=[
            {"role": "system", "content": link_system_prompt},
            {"role": "user", "content": get_links_user_prompt(url)}
        ],
        response_format={"type": "json_object"}
    )
    result = response.choices[0].message.content
    links = json.loads(result)
    return links
    

In [12]:
select_relevant_links("https://www.dintaifung.com.tw/store.php")

{'links': [{'type': 'about page',
   'url': 'https://www.dintaifung.com.tw/about.php'},
  {'type': 'careers page',
   'url': 'https://www.104.com.tw/jobbank/custjob/index.php?r=cust&j=4c70436d485c3e6848323c1d1d1d1d5f2443a363189j99'}]}

In [15]:
def select_relevant_links(url):
    print(f"Selecting relevant links for {url} by calling {non_reasoning_model}")
    response = openai.chat.completions.create(
        model=non_reasoning_model,
        messages=[
            {"role": "system", "content": link_system_prompt},
            {"role": "user", "content": get_links_user_prompt(url)}
        ],
        response_format={"type": "json_object"}
    )
    result = response.choices[0].message.content
    links = json.loads(result)
    print(f"Found {len(links['links'])} relevant links")
    return links

In [16]:
select_relevant_links("https://www.dintaifung.com.tw/store.php")

Selecting relevant links for https://www.dintaifung.com.tw/store.php by calling grok-4-1-fast-non-reasoning
Found 2 relevant links


{'links': [{'type': 'about page',
   'url': 'https://www.dintaifung.com.tw/about.php'},
  {'type': 'careers page',
   'url': 'https://www.104.com.tw/jobbank/custjob/index.php?r=cust&j=4c70436d485c3e6848323c1d1d1d1d5f2443a363189j99'}]}

In [10]:
select_relevant_links("https://huggingface.co")

Selecting relevant links for https://huggingface.co by calling gpt-5-nano
Found 3 relevant links


{'links': [{'type': 'brand page', 'url': 'https://huggingface.co/brand'},
  {'type': 'careers page', 'url': 'https://apply.workable.com/huggingface/'},
  {'type': 'company page',
   'url': 'https://www.linkedin.com/company/huggingface/'}]}

## Second step: make the brochure!

Assemble all the details into another prompt to GPT-5-nano

In [17]:
def fetch_page_and_all_relevant_links(url):
    contents = fetch_website_contents(url)
    relevant_links = select_relevant_links(url)
    result = f"## Landing Page:\n\n{contents}\n## Relevant Links:\n"
    for link in relevant_links['links']:
        result += f"\n\n### Link: {link['type']}\n"
        result += fetch_website_contents(link["url"])
    return result

In [18]:
print(fetch_page_and_all_relevant_links("https://huggingface.co"))

Selecting relevant links for https://huggingface.co by calling grok-4-1-fast-non-reasoning
Found 3 relevant links
## Landing Page:

Hugging Face – The AI community building the future.

Hugging Face
Models
Datasets
Spaces
Community
Docs
Enterprise
Pricing
Log In
Sign Up
The AI community building the future.
The platform where the machine learning community collaborates on models, datasets, and applications.
Explore AI Apps
or
Browse 1M+ models
Trending on
this week
Models
tencent/HunyuanVideo-1.5
Updated
1 day ago
•
1.68k
•
666
facebook/sam3
Updated
5 days ago
•
115k
•
646
Supertone/supertonic
Updated
4 days ago
•
4.11k
•
265
facebook/sam-3d-objects
Updated
4 days ago
•
182
WeiboAI/VibeThinker-1.5B
Updated
1 day ago
•
16.6k
•
473
Browse 1M+ models
Spaces
Running
on
Zero
MCP
Featured
1.27k
Qwen Image Edit Camera Control
🎬
1.27k
Fast 4 step inference with Qwen Image Edit 2509
Running
on
CPU Upgrade
Featured
2.42k
The Smol Training Playbook
📚
2.42k
The secrets to building world-class LLMs

In [19]:
brochure_system_prompt = """
You are an assistant that analyzes the contents of several relevant pages from a company website
and creates a short brochure about the company for prospective customers, investors and recruits.
Respond in markdown without code blocks.
Include details of company culture, customers and careers/jobs if you have the information.
"""

# Or uncomment the lines below for a more humorous brochure - this demonstrates how easy it is to incorporate 'tone':

# brochure_system_prompt = """
# You are an assistant that analyzes the contents of several relevant pages from a company website
# and creates a short, humorous, entertaining, witty brochure about the company for prospective customers, investors and recruits.
# Respond in markdown without code blocks.
# Include details of company culture, customers and careers/jobs if you have the information.
# """


In [20]:
def get_brochure_user_prompt(company_name, url):
    user_prompt = f"""
You are looking at a company called: {company_name}
Here are the contents of its landing page and other relevant pages;
use this information to build a short brochure of the company in markdown without code blocks.\n\n
"""
    user_prompt += fetch_page_and_all_relevant_links(url)
    user_prompt = user_prompt[:5_000] # Truncate if more than 5,000 characters
    return user_prompt

In [21]:
get_brochure_user_prompt("HuggingFace", "https://huggingface.co")

Selecting relevant links for https://huggingface.co by calling grok-4-1-fast-non-reasoning
Found 4 relevant links


'\nYou are looking at a company called: HuggingFace\nHere are the contents of its landing page and other relevant pages;\nuse this information to build a short brochure of the company in markdown without code blocks.\n\n\n## Landing Page:\n\nHugging Face – The AI community building the future.\n\nHugging Face\nModels\nDatasets\nSpaces\nCommunity\nDocs\nEnterprise\nPricing\nLog In\nSign Up\nThe AI community building the future.\nThe platform where the machine learning community collaborates on models, datasets, and applications.\nExplore AI Apps\nor\nBrowse 1M+ models\nTrending on\nthis week\nModels\ntencent/HunyuanVideo-1.5\nUpdated\n1 day ago\n•\n1.68k\n•\n666\nfacebook/sam3\nUpdated\n5 days ago\n•\n115k\n•\n646\nSupertone/supertonic\nUpdated\n4 days ago\n•\n4.11k\n•\n265\nfacebook/sam-3d-objects\nUpdated\n4 days ago\n•\n182\nWeiboAI/VibeThinker-1.5B\nUpdated\n1 day ago\n•\n16.6k\n•\n473\nBrowse 1M+ models\nSpaces\nRunning\non\nZero\nMCP\nFeatured\n1.27k\nQwen Image Edit Camera Contro

In [22]:
def create_brochure(company_name, url):
    response = openai.chat.completions.create(
        model=non_reasoning_model,
        messages=[
            {"role": "system", "content": brochure_system_prompt},
            {"role": "user", "content": get_brochure_user_prompt(company_name, url)}
        ],
    )
    result = response.choices[0].message.content
    display(Markdown(result))

In [23]:
create_brochure("HuggingFace", "https://huggingface.co")

Selecting relevant links for https://huggingface.co by calling grok-4-1-fast-non-reasoning
Found 4 relevant links


# Discover Hugging Face  
**The AI Community Building the Future**

## Welcome to the Home of Machine Learning
Hugging Face is the world's leading platform where the machine learning community collaborates on **1M+ models**, **250k+ datasets**, and **400k+ applications (Spaces)**. Explore trending AI apps, from video generation (e.g., HunyuanVideo-1.5, Dream-wan2) to image editing (Qwen Image Edit) and depth mapping (Depth Anything 3). Host unlimited public models, datasets, and apps across text, image, video, audio, and 3D modalities. Build your ML portfolio, share work globally, and accelerate innovation with our open-source stack.

## For Enterprises & Teams
Scale securely with **Enterprise Hub** and **Team plans**:
- **Team**: Starting at **$20/user/month** – SSO, audit logs, resource groups, analytics, 5x ZeroGPU quota, private storage (1TB/member), priority support.
- **Enterprise**: Custom contracts – Advanced compute (ZeroGPU), inference providers, regions, token management, and billing controls.
- **PRO**: **$9/month** for individuals – 10x storage, 20x inference credits, Dev Mode Spaces.

Join forward-thinking organizations powering AI at scale.

## Collaborative Community Culture
We're a vibrant, open ecosystem fostering creation, discovery, and collaboration. Move faster together – from solo builders to global teams – with docs, forums, and shared resources. Everyone from hobbyists to enterprises thrives here.

## Careers at Hugging Face
Build the future of AI in a dynamic, community-driven environment. While specific openings aren't listed, contribute via models, datasets, Spaces, or join our enterprise team for roles in engineering, support, and platform innovation. Showcase your work and grow your ML profile!

**Ready to join the revolution?**  
[Sign Up](https://huggingface.co/join) | [Explore Models](https://huggingface.co/models) | [Enterprise](https://huggingface.co/enterprise) | [Pricing](https://huggingface.co/pricing)

## Finally - a minor improvement

With a small adjustment, we can change this so that the results stream back from OpenAI,
with the familiar typewriter animation

In [24]:
def stream_brochure(company_name, url):
    stream = openai.chat.completions.create(
        model=non_reasoning_model,
        messages=[
            {"role": "system", "content": brochure_system_prompt},
            {"role": "user", "content": get_brochure_user_prompt(company_name, url)}
          ],
        stream=True
    )    
    response = ""
    display_handle = display(Markdown(""), display_id=True)
    for chunk in stream:
        response += chunk.choices[0].delta.content or ''
        update_display(Markdown(response), display_id=display_handle.display_id)

In [25]:
stream_brochure("HuggingFace", "https://huggingface.co")

Selecting relevant links for https://huggingface.co by calling grok-4-1-fast-non-reasoning
Found 3 relevant links


# Discover Hugging Face  
**The AI Community Building the Future**

## Who We Are
Hugging Face is the world's leading collaboration platform for the machine learning community. We empower creators to **host, share, discover, and experiment** with open-source ML models, datasets, and applications. With over **1M+ models**, **400k+ Spaces (AI apps)**, and **250k+ datasets**, we're the central hub where the community collaborates to advance AI across text, image, video, audio, and 3D modalities.

**Our Mission**: Build an open and ethical AI future together, accelerating innovation for machine learning engineers, scientists, and end-users.

## What We Offer
- **Explore Trending AI**: Dive into top models like facebook/sam3, tencent/HunyuanVideo-1.5, and apps like Qwen Image Edit or Depth Anything 3.
- **Open-Source Stack**: Move faster with unlimited public hosting, ZeroGPU compute, and tools for all modalities.
- **Build Your Portfolio**: Share work globally and grow your ML profile.

## For Enterprises & Teams
Scale securely with our **Enterprise Hub**:
- SSO, audit logs, resource groups, token management, and analytics.
- Advanced compute (ZeroGPU boosts), private storage (1TB+ per member), and inference providers.
- Priority support, managed billing, and custom contracts.
- **Team plans** start at $20/user/month; contact sales for Enterprise.

Join forward-thinking organizations leveraging Hugging Face for production-grade AI.

## Join Our Community & Careers
We're a vibrant, collaborative culture focused on open-source innovation and ethical AI.  
**Current Openings**: Explore exciting roles at [Hugging Face Careers](careers-link) to contribute to the future of ML. Build with us—sign up today!

**[Sign Up Free](signup-link) | [Enterprise](enterprise-link) | [Browse Models](models-link)**

*Hugging Face: Where the ML community creates, collaborates, and accelerates.*  
*Colors: #FFD21E (Gold) | #FF9D00 (Orange) | #6B7280 (Gray)*

In [26]:

stream_brochure("鼎泰豐", "https://www.dintaifung.com.tw/store.php")

Selecting relevant links for https://www.dintaifung.com.tw/store.php by calling grok-4-1-fast-non-reasoning
Found 2 relevant links


# Discover Din Tai Fung  
**鼎泰豐** – World-Renowned Masters of Xiao Long Bao  

![Din Tai Fung Logo](https://example.com/dintaifung-logo) *(Iconic golden 18-pleat dumplings)*  

## Our Story  
Founded in 1958 in Taiwan by Mr. Yang Bingyi, Din Tai Fung began as a modest cooking oil shop in Taipei's Xinyi district. Facing declining oil sales in the 1970s, the Yang family pivoted to handmade noodles and xiao long bao (soup dumplings). What started as a neighborhood eatery evolved into a global phenomenon, famed for its **thin-skinned, juicy, 18-pleat xiao long bao** made with natural ingredients, handmade dough, and generous fillings.  

Today, Din Tai Fung operates **dozens of locations worldwide**, including Taiwan, Malaysia (e.g., Sunway Velocity, KLIA2), Singapore (VivoCity), Japan, and more – conquering taste buds everywhere.  

## Our Core Beliefs  
- **Perfect Details in Service**: Genuine, proactive care tailored to every guest.  
- **Sustainable Success**: No short-term gains; enterprise-level management for global excellence.  
- **Quality is Life**: Rigorous ingredient selection, precise cooking, and flawless presentation.  
- **Brand as Responsibility**: Every bite delivers trust, safety, and joy (*Anxin Declaration*).  

*Recent Highlights*: 2025 awards include 500 Plates Best Service Team, Hall of Fame, and Sweet awards; new openings in Malaysia and Singapore.  

## Why Din Tai Fung?  
**For Customers**: Indulge in absolute culinary perfection – from signature xiao long bao to noodles and dim sum. Use our Waitlist APP for seamless queuing at bustling stores worldwide. Shop online for gourmet delights.  

**For Investors**: A legacy of resilience and expansion, from alleyway roots to international acclaim. Proven model blending tradition with modern operations for enduring growth.  

**For Recruits**: Join a passionate team upholding family values and world-class standards. Explore **Talent Recruitment** opportunities in service, culinary arts, and management. Build your career in a culture of sincerity, precision, and global impact. *Careers page highlights ongoing hiring for dedicated talents.*  

**Visit Us**: [Stores](locations) | [Online Shop](shop) | [Contact](contact)  
*鼎泰豐 – Where Every Pleat Tells a Story of Excellence.*  

*Languages: 繁體中文 | English | 日本語 | 한국어*

<table style="margin: 0; text-align: left;">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../assets/business.jpg" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#181;">Business applications</h2>
            <span style="color:#181;">In this exercise we extended the Day 1 code to make multiple LLM calls, and generate a document.

This is perhaps the first example of Agentic AI design patterns, as we combined multiple calls to LLMs. This will feature more in Week 2, and then we will return to Agentic AI in a big way in Week 8 when we build a fully autonomous Agent solution.

Generating content in this way is one of the very most common Use Cases. As with summarization, this can be applied to any business vertical. Write marketing content, generate a product tutorial from a spec, create personalized email content, and so much more. Explore how you can apply content generation to your business, and try making yourself a proof-of-concept prototype. See what other students have done in the community-contributions folder -- so many valuable projects -- it's wild!</span>
        </td>
    </tr>
</table>

<table style="margin: 0; text-align: left;">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../assets/important.jpg" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#900;">Before you move to Week 2 (which is tons of fun)</h2>
            <span style="color:#900;">Please see the week1 EXERCISE notebook for your challenge for the end of week 1. This will give you some essential practice working with Frontier APIs, and prepare you well for Week 2.</span>
        </td>
    </tr>
</table>

<table style="margin: 0; text-align: left;">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../assets/resources.jpg" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#f71;">A reminder on 3 useful resources</h2>
            <span style="color:#f71;">1. The resources for the course are available <a href="https://edwarddonner.com/2024/11/13/llm-engineering-resources/">here.</a><br/>
            2. I'm on LinkedIn <a href="https://www.linkedin.com/in/eddonner/">here</a> and I love connecting with people taking the course!<br/>
            3. I'm trying out X/Twitter and I'm at <a href="https://x.com/edwarddonner">@edwarddonner<a> and hoping people will teach me how it's done..  
            </span>
        </td>
    </tr>
</table>

<table style="margin: 0; text-align: left;">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../assets/thankyou.jpg" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#090;">Finally! I have a special request for you</h2>
            <span style="color:#090;">
                My editor tells me that it makes a MASSIVE difference when students rate this course on Udemy - it's one of the main ways that Udemy decides whether to show it to others. If you're able to take a minute to rate this, I'd be so very grateful! And regardless - always please reach out to me at ed@edwarddonner.com if I can help at any point.
            </span>
        </td>
    </tr>
</table>